In [2]:
from settings import * 

In [3]:
with open(DRMM_PK_FPATH) as f:
    data_pickle = pk.load(f)

In [4]:
data_pickle.keys()

['qid_docid2histvec',
 'n_pos',
 'instances',
 'candidates',
 'QUERIES',
 'relevance']

In [5]:
QUERIES = data_pickle['QUERIES']
instances = data_pickle['instances']
candidates = data_pickle['candidates']
qid_docid2histvec = data_pickle['qid_docid2histvec']

In [22]:
relevance = data_pickle['relevance']

In [6]:
def idf(para):
    return -10 if para==PARA_PLACEHOLDER else 1.0 
IDFs = {}
for qid in QUERIES.keys(): 
    IDFs[qid] = np.array([idf(para) for para in QUERIES[qid]])

In [7]:
data_pickle['IDFs'] = IDFs

In [8]:
import DRMM
reload(DRMM)
from DRMM import *

Using TensorFlow backend.


In [9]:
ffwd_3layer = Sequential(
    [Dense(input_dim= N_HISTBINS, output_dim=10, activation='relu'),
     Dense(input_dim= N_HISTBINS, output_dim=5, activation='relu'),
     Dense(output_dim=1, activation='tanh'),
     ], 
    name='ffwd_3layer')

In [10]:
ffwd_4layer = Sequential(
    [Dense(input_dim= N_HISTBINS, output_dim=10, activation='relu'),
     Dense(input_dim= N_HISTBINS, output_dim=6, activation='relu'),
     Dense(input_dim= N_HISTBINS, output_dim=3, activation='relu'),
     Dense(output_dim=1, activation='tanh'),
     ], 
    name='ffwd_4layer')

In [11]:
from utils import KFold

In [12]:
MAX_QLEN = 12

In [15]:
def tune_ffwd(feed_forward, suffix = '', instances = instances):
    scoring_model, ranking_model = gen_DRMM_model(MAX_QLEN, feed_forward)
    initial_weights = ranking_model.get_weights()
    KFold(ranking_model=ranking_model, scoring_model=scoring_model, data_pickle=data_pickle,
          initial_weights = initial_weights, verbose=0,
          fpath = '../data/trec-output/0203_5fold_%s.rankedlist' % suffix, 
          instances = instances)
    KFold(ranking_model=ranking_model, scoring_model=scoring_model, data_pickle=data_pickle,
          initial_weights = initial_weights, verbose=0, K = 30,
          fpath = '../data/trec-output/0203_LOO_%s.rankedlist' % suffix, 
          instances = instances)

In [16]:
tune_ffwd(ffwd_3layer, 'ffwd3layer')

fold 0 [14, 16, 15, 6, 1, 26]
fold 0 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer.rankedlist...
fold 1 [22, 17, 30, 10, 25, 2]
fold 1 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer.rankedlist...
fold 2 [18, 24, 8, 19, 29, 13]
fold 2 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer.rankedlist...
fold 3 [28, 27, 7, 5, 23, 12]
fold 3 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer.rankedlist...
fold 4 [4, 11, 3, 20, 21, 9]
fold 4 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer.rankedlist...
fold 0 [15]
fold 0 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer.rankedlist...
fold 1 [21]
fold 1 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer.rankedlist...
fold 2 [24]
fold 2 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer.rankedlist...
fold 3 [25]
fold 3 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer.rankedlist...
fold 4 [3]
fold 4 complete

In [17]:
tune_ffwd(ffwd_4layer, 'ffwd4layer')

fold 0 [24, 19, 8, 26, 14, 11]
fold 0 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer.rankedlist...
fold 1 [5, 29, 9, 10, 4, 16]
fold 1 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer.rankedlist...
fold 2 [13, 12, 17, 28, 21, 18]
fold 2 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer.rankedlist...
fold 3 [20, 3, 22, 1, 23, 2]
fold 3 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer.rankedlist...
fold 4 [6, 25, 15, 27, 7, 30]
fold 4 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer.rankedlist...
fold 0 [8]
fold 0 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer.rankedlist...
fold 1 [7]
fold 1 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer.rankedlist...
fold 2 [12]
fold 2 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer.rankedlist...
fold 3 [4]
fold 3 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer.rankedlist...
fold 4 [15]
fold 4 complete, 

In [23]:
N_PAIRS_PER_QUERY = 8000 # the smallest query (22) have 8816 possible pairs 
from numpy.random import choice 
from collections import defaultdict
instances2 = {} # mapping qid to list, instances[qid] = list (pos_docid, neg_docid) pairs for qid, 

np.random.seed(1)
for qid in QUERIES.keys():
    rel_scores = defaultdict(list) # mapping a rel score to list of docids
    for docid in candidates[qid]:
        rel = relevance[(qid,docid)]
        rel_scores[rel].append(docid)
    scores = sorted( rel_scores.keys(), reverse=True ) # scores are sorted in desc order
    print 'scores =',scores, 
    
    all_instances = []
    for i in xrange(len(scores)): 
        pos_score = scores[i]
        for j in xrange(i+1, len(scores)): 
            neg_score = scores[j]
            for posid in rel_scores[pos_score]:
                for negid in rel_scores[neg_score]: 
                    all_instances.append( (posid, negid) )
    
    instances_for_q = []
    for i in choice(len(all_instances), N_PAIRS_PER_QUERY, replace=False):
        instances_for_q.append(all_instances[i])
    
    print 'got %d instances out of %d for query %d' % (len(instances_for_q), len(all_instances), qid)
    instances2[qid] = instances_for_q

scores = [2, 1, 0] got 8000 instances out of 163263 for query 1
scores = [2, 1, 0] got 8000 instances out of 42372 for query 2
scores = [2, 1, 0] got 8000 instances out of 192356 for query 3
scores = [2, 1, 0] got 8000 instances out of 24345 for query 4
scores = [2, 1, 0] got 8000 instances out of 126691 for query 5
scores = [2, 1, 0] got 8000 instances out of 97817 for query 6
scores = [2, 1, 0] got 8000 instances out of 72479 for query 7
scores = [2, 1, 0] got 8000 instances out of 435854 for query 8
scores = [2, 1, 0] got 8000 instances out of 126334 for query 9
scores = [2, 0] got 8000 instances out of 21318 for query 10
scores = [2, 1, 0] got 8000 instances out of 255176 for query 11
scores = [2, 1, 0] got 8000 instances out of 117927 for query 12
scores = [2, 1, 0] got 8000 instances out of 189428 for query 13
scores = [2, 1, 0] got 8000 instances out of 124466 for query 14
scores = [2, 1, 0] got 8000 instances out of 68651 for query 15
scores = [2, 1, 0] got 8000 instances out o

In [25]:
tune_ffwd(ffwd_3layer, 'ffwd3layer_unif', instances=instances2)

fold 0 [12, 28, 17, 11, 24, 25]
fold 0 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer_unif.rankedlist...
fold 1 [1, 21, 30, 10, 6, 19]
fold 1 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer_unif.rankedlist...
fold 2 [18, 2, 29, 20, 7, 13]
fold 2 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer_unif.rankedlist...
fold 3 [27, 15, 5, 14, 23, 16]
fold 3 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer_unif.rankedlist...
fold 4 [26, 4, 8, 3, 22, 9]
fold 4 complete, outputting to ../data/trec-output/0203_5fold_ffwd3layer_unif.rankedlist...
fold 0 [17]
fold 0 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer_unif.rankedlist...
fold 1 [22]
fold 1 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer_unif.rankedlist...
fold 2 [2]
fold 2 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer_unif.rankedlist...
fold 3 [6]
fold 3 complete, outputting to ../data/trec-output/0203_LOO_ffwd3layer_un

In [26]:
tune_ffwd(ffwd_4layer, 'ffwd4layer_unif', instances=instances2)

fold 0 [2, 20, 29, 25, 12, 4]
fold 0 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer_unif.rankedlist...
fold 1 [14, 7, 9, 10, 26, 28]
fold 1 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer_unif.rankedlist...
fold 2 [13, 16, 21, 27, 22, 18]
fold 2 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer_unif.rankedlist...
fold 3 [3, 8, 1, 24, 23, 19]
fold 3 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer_unif.rankedlist...
fold 4 [11, 6, 17, 15, 5, 30]
fold 4 complete, outputting to ../data/trec-output/0203_5fold_ffwd4layer_unif.rankedlist...
fold 0 [29]
fold 0 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer_unif.rankedlist...
fold 1 [5]
fold 1 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer_unif.rankedlist...
fold 2 [16]
fold 2 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer_unif.rankedlist...
fold 3 [26]
fold 3 complete, outputting to ../data/trec-output/0203_LOO_ffwd4layer_u